In [ ]:
import os
import sqlite3
import pandas as pd
import requests
import plotly.express as px

### pyth data


In [ ]:
PYTH_API_URL = "https://benchmarks.pyth.network/v1/"
PYTH_ENDPOINT = "shims/tradingview/history"
PYTH_ASSETS = [
    "Crypto.BONK/USD",
    "Crypto.LINK/USD",
    "Crypto.AVAX/USD",
    "Crypto.INJ/USD",
    "Crypto.PEPE/USD",
]
PYTH_START_TIMESTAMP = 1704085200
PYTH_END_TIMESTAMP = 1706677200

df_pyth_prices = pd.DataFrame()
for asset in PYTH_ASSETS:
    last_timestamp = PYTH_START_TIMESTAMP
    while last_timestamp < PYTH_END_TIMESTAMP:
        params = {
            "symbol": asset,
            "resolution": 5,
            "from": last_timestamp,
            "to": last_timestamp + (2000 * 60 * 5),
        }
        response = requests.get(f"{PYTH_API_URL}{PYTH_ENDPOINT}", params=params)

        if response.status_code == 200:
            data = response.json()
            if data["s"] == "ok":
                data.pop("s")
                df = pd.DataFrame.from_dict(
                    data,
                )
                print(
                    f"Successfully fetched {asset} data from {last_timestamp} to {df['t'].max()}"
                )
                df["asset"] = asset
                df_pyth_prices = pd.concat([df_pyth_prices, df])
                last_timestamp = df["t"].max()
            else:
                print(f"Error fetching data: {data}")

        else:
            print(f"Error fetching data: {response.status_code}")
            print(f"Error: {response.text}")

In [ ]:
for asset in PYTH_ASSETS:
    start_time = df_pyth_prices[df_pyth_prices["asset"] == asset]["t"].min()
    end_time = df_pyth_prices[df_pyth_prices["asset"] == asset]["t"].max()

    # convert to readable dates
    print(
        f"{asset}: {pd.to_datetime(start_time, unit='s')} - {pd.to_datetime(end_time, unit='s')}"
    )

In [ ]:
df_pyth_prices.to_csv("../../data/ml_export_pyth_prices.csv", index=False)